In [91]:
from random import choices, choice
import base64
from Crypto.Cipher import AES

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]
        
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * npad

def UnPadPlaintext(plaintext: bytes, blocksize = 16):
    assert type(plaintext) is bytes and len(plaintext) % blocksize == 0
    final = plaintext[-1]
    assert all( char == final for char in plaintext[-final:])
    return plaintext[:-final]

def EncryptECB(plaintext, key):
    BLOCKSIZE = 16
    assert len(key) == BLOCKSIZE
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    ECBcipher = AES.new(key, AES.MODE_ECB)
    blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cypher_blocks = [ ECBcipher.encrypt(block) for block in blocks ]
    return b''.join( ECBcipher.encrypt(block) for block in blocks )

def DecryptECB(ciphertext, key):
    ECBcipher = AES.new(key, AES.MODE_ECB)
    blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
    plaintext = b''.join( ECBcipher.decrypt(block) for block in blocks )
    return UnPadPlaintext(plaintext)

KEYSIZE = 16

In [44]:
def Parser(s: bytes):
    out = dict()
    for pair in s.split(b'&'):
        key, value = pair.split(b'=')
        out[key] = value
    return out

In [45]:
Parser(b'foo=bar&baz=qux&zap=zazzle')

{b'foo': b'bar', b'baz': b'qux', b'zap': b'zazzle'}

In [46]:
def profile_for(email: bytes):
    assert type(email) is bytes and b'&' not in email and b'=' not in email
    uid = bytes( choices(range(48, 58), k = 6) )
    role = b'user'
    return b'email=' + email + b'&uid=' + uid + b'&role=' + role

In [47]:
email = b'foo@bar.com'
profile_for(email)

b'email=foo@bar.com&uid=824078&role=user'

In [80]:
# Make functions to create encrypted profile
# And decrypt it

def encrypted_profile_for(email: bytes):
    # This function will be available to the attacker, but we will pretend they can only use the function
    # but not see the key used inside it
    key = bytes([195, 179, 239, 34, 91, 179, 74, 150, 151, 38, 120, 53, 134, 233, 178, 193])  # a random static key
    plaintext = profile_for(email)
    return EncryptECB(plaintext, key)

def decrypt_profile(ciphertext: bytes):
    assert type(ciphertext) is bytes
    key = bytes([195, 179, 239, 34, 91, 179, 74, 150, 151, 38, 120, 53, 134, 233, 178, 193])  # a random static key
    return DecryptECB(ciphertext, key = key)

In [38]:
# The instructions are a little unclear here... but here is my interpretation
# From here on out the code will be from the perspective of the "attacker"
# They will be able to use `profile_for` and `encrypted_profile_for`
# BUT they will NOT be able to see inside the funtions and see what key is being used

In [50]:
# This is all that's available, not the interior
profile_for

<function __main__.profile_for(email: bytes)>

In [53]:
# This is all that's available, not the interior
encrypted_profile_for

<function __main__.encrypted_profile_for(email: bytes)>

In [59]:
# Ok so from the perspective of the attacker, I don't know how this function works

for n in range(10):
    print(profile_for(b'foo@bar.com'))


b'email=foo@bar.com&uid=286020&role=user'
b'email=foo@bar.com&uid=877491&role=user'
b'email=foo@bar.com&uid=690723&role=user'
b'email=foo@bar.com&uid=460796&role=user'
b'email=foo@bar.com&uid=352246&role=user'
b'email=foo@bar.com&uid=855241&role=user'
b'email=foo@bar.com&uid=520518&role=user'
b'email=foo@bar.com&uid=336084&role=user'
b'email=foo@bar.com&uid=438639&role=user'
b'email=foo@bar.com&uid=879660&role=user'


In [60]:
# So, looks like the order is always email, uid, role
# And the uid is always a six-digit number

In [66]:
# So first I need to get a cipherblock for just "admin" (padded)
# "email=" has 6 chars
# "admin" has 5 chars
# so I need an email that has 10 chars preceding admin, and then 11 chars padding after admin

BLOCKSIZE = 16 # As the attacker I think I can fairly safely assume this blocksize

email = b'A'*10 + b'admin' + b'\x0b'*11 + b'@bar.com'
print(email)

b'AAAAAAAAAAadmin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b@bar.com'


In [70]:
# So, the second block of this cipher text should be equal to "admin" (padded)
ciphertext = encrypted_profile_for(email)
print(ciphertext)
admin_block = ciphertext[ BLOCKSIZE:2*BLOCKSIZE ]
print(admin_block)

b'\x8a\xf9\x1e\x02\x0c\x01\x8dBe\x00\xc8\xc8\xf9\x92:6\xc5\xff\x1b\xf3p\xa6\xb3@9\xfd\xf1`(\xe8\xfe\x81\x17L6.\xfe\xab\xd7E\xe5\x87,G\xb0~\xe8\x0e\xe4\xe8\xd4\x085.\xa2\x82k\xef\x7f\xac\xaa\xa9\xd5?'
b'\xc5\xff\x1b\xf3p\xa6\xb3@9\xfd\xf1`(\xe8\xfe\x81'


In [74]:
# So now I need a ciphertext such that the penultimate block ends with "role=" so that 
# So that "user" (padded) is in a block all by itself
# I'll construct an email such that len(plaintext) % BLOCKSIZE == len("user")

email = b'foo@bar.com'
plaintext = profile_for(email)

while len(plaintext) % BLOCKSIZE != len(b'user'):
    email = b'A' + email
    plaintext = profile_for(email)

print(email)

b'AAAAAAAAAAAAAAfoo@bar.com'


In [85]:
# So this email should produce a ciphertext with "user" alone in the last block
ciphertext = encrypted_profile_for(email)
print(ciphertext)

b'\x8a\xf9\x1e\x02\x0c\x01\x8dBe\x00\xc8\xc8\xf9\x92:6\x04\x80\xbf\x08^\xc34x\x9a\xb0\xea\x15\x99z\x8f\xba\x16\xee\x92\xc4\xd4Aj\xbe\xe0\xfa\xf0\x97) \xd6\x96\x08Uu\x8b=\xc3;\x03#3\xcb;\xeb"J\x8b'


In [86]:
# And now I just need to replace that last block with the admin_block

new_ciphertext = ciphertext[:-BLOCKSIZE] + admin_block

In [93]:
# And this should decrypt back to an admin profile
decrypt_profile(new_ciphertext)

b'email=AAAAAAAAAAAAAAfoo@bar.com&uid=584585&role=admin'

In [94]:
# GREAT SUCCESS